In [186]:
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
from sklearn import linear_model
import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn import ensemble
import time
import random

In [84]:
mydata=pd.read_csv('/Users/s0c01xa/Documents/label_imbalance.csv')

In [14]:
#x=mydata.iloc[:,0:5]
#y=mydata['Newborn']

In [102]:
#Class imbalance problem
mynewdata=mydata.iloc[:,0:6]
data1 = mynewdata.loc[mynewdata['Newborn']==0]
data2=mynewdata.loc[mynewdata['Newborn']==1]
df=data1.iloc[:,0:6]

In [104]:
len(mydata)-len(data1)

1850

# CLUSTEERING USING K-MEANS AND ENSURING ALL THE CLUSTERS HAVE SIMILAR NO. OF ELEMENTS


In [105]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=1).fit(df.iloc[:,0:5])

In [108]:
#Labels it will store the entire labels for each of the 
clust_labels=kmeans.labels_

In [112]:
#len(df[clust_labels==1])
a=df[clust_labels==0]
#a.append(data2)


# FIT LOGISTIC REGRESSION TO EACH CLUSTER

In [127]:
import pickle
from sklearn.linear_model import LogisticRegression as lg

model=lg(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None)

#Saving the models to predict the test data
for i in range(0,5):
    feat=df[clust_labels==i]
    feat=feat.append(data2)
    x=feat.iloc[:,0:5]
    y=feat['Newborn']
    model.fit(x,y)
    filename = "/Users/s0c01xa/Documents/pkl_models_im/model"+'_'+str(i+1)+'.pkl'
    #Open the filename to write 'w'
    file_object = open(filename, 'w')
    #dumps the model by the file name in pickle
    s = pickle.dump(model, file_object)
    #close the file
    file_object.close()
    feat=[]

In [151]:
'''feat=df[clust_labels==1]
feat=feat.append(data2)
x=feat.iloc[:,0:5]
y=feat['Newborn']
model.fit(x,y)
cross_val_score(model,x,y,cv=10)'''

"feat=df[clust_labels==1]\nfeat=feat.append(data2)\nx=feat.iloc[:,0:5]\ny=feat['Newborn']\nmodel.fit(x,y)\ncross_val_score(model,x,y,cv=10)"

# PREDICTING TEST USING SAVED MODELS 

STEP1- PREDICTING FROM ALL THE MODELS AND ENSEMBLE OUT OF THEM- MAX/AVG

In [153]:
test_data=pd.read_csv('/Users/s0c01xa/Documents/test_imbalance.csv')


In [157]:
#Defining the test data
test_x=test_data.iloc[:,0:5]
test_y=test_data['Newborn']


In [164]:
test_x=np.array(test_x)


In [168]:
#Now predicting from all the saved models
#for j in range(0,len(test_x)):

all_pred_Y = np.zeros((len(test_x), 5))

for i in range(0,5):
    #x_t=test_x[j]
    #model.fit(x,y)
    filename = "/Users/s0c01xa/Documents/pkl_models_im/model"+'_'+str(i+1)+'.pkl'
    #Open the filename to write 'w'
    file_object = open(filename, 'r')
    #dumps the model
    classifier = pickle.load(file_object)
    #close the file
    pred_Y = classifier.predict(test_x)
    all_pred_Y[:,i] = pred_Y
    file_object.close()
    
#all_pred_Y saves row wise the prediction
#Now finding the max in each row as the final prediction

        
   

array([[ 0.,  1.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  1.,  1.],
       [ 1.,  1.,  0.,  1.,  1.],
       ..., 
       [ 1.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  1.,  0.,  1.],
       [ 0.,  1.,  0.,  1.,  0.]])

In [177]:
y_pred=np.zeros(len(all_pred_Y))
for i in range(0,len(all_pred_Y )):
    if (np.count_nonzero(all_pred_Y [i]==1)>np.count_nonzero(all_pred_Y [0]==0)):
        y_pred[i]=1
    else :
        y_pred[i]=0


In [185]:
z=y_pred-test_y
np.true_divide(np.count_nonzero(z==0),len(test_y))

0.628